# LSTM MODEL

In [0]:
import json
import numpy as np
import re
import io
import nltk
import h5py
from keras import backend as K
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout, RepeatVector, Activation, merge, Lambda, Flatten, Reshape
from keras.layers import LSTM, Bidirectional, TimeDistributed, GRU
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.layers import concatenate

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import os
BASE_PATH = '/gdrive/My Drive/colab_files/project/'
if not os.path.exists(BASE_PATH):
    os.makedirs(BASE_PATH)
DATA_PATH = '/content/'

if not os.path.exists(os.path.join(DATA_PATH, 'glove.6B.100d.txt')):
    os.chdir(BASE_PATH)
    !wget http://nlp.stanford.edu/data/glove.6B.zip
    !unzip glove.6B.zip
    !rm glove.6B.zip
    !cp glove.6B.100d.txt /content
    # !cp harry_potter.txt /content
os.chdir('/content')

In [0]:
!cp glove.6B.100d.txt '/content/drive/My Drive/dl_proj_data/'

In [0]:
embeddings_index = {}
f = open('/content/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
context = h5py.File('/content/drive/My Drive/dl_proj_data/context.h5','r')
questions = h5py.File('/content/drive/My Drive/dl_proj_data/questions.h5','r')
answers = h5py.File('/content/drive/My Drive/dl_proj_data/answers.h5','r')
ans_begin = h5py.File('/content/drive/My Drive/dl_proj_data/begin.h5','r')
ans_end = h5py.File('/content/drive/My Drive/dl_proj_data/end.h5','r')

In [0]:
c_data = context['context'][:]
qn_data = questions['questions'][:]
ans_data = answers['answers'][:]

In [0]:
begin_ans = ans_begin['begin'][:]
end_ans = ans_end['end'][:]

In [0]:
# loding vocabulary
word_index = np.load('/content/drive/My Drive/dl_proj_data/words.npy', allow_pickle=True).item()

In [0]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
vocab_size = len(word_index) + 1
#embedding_vector_length = 50
batch = 128
max_span_begin = np.amax(begin_ans)
max_span_end = np.amax(end_ans)
slce = 10000


In [0]:
print("Vocab Size")
vocab_size

Vocab Size


119616

In [0]:
context_input = Input(shape=(700, ), dtype='int32', name='c_data')
context_embed = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], 
              input_length=700, trainable=False)(context_input)
#lstm_out = (LSTM(256, return_sequences=True, implementation=2))(x)
drop_1 = Dropout(0.5)(context_embed)
#drop_1 = Dropout(0.5)(lstm_out)











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
ques_input = Input(shape=(100, ), dtype='int32', name='qn_data')
question_embed = Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], 
              input_length=100, trainable=False)(ques_input)
#lstm_out = (LSTM(256, return_sequences=True, implementation=2))(x)
drop_2 = Dropout(0.5)(question_embed)
#drop_2 = Dropout(0.5)(lstm_out)

In [0]:
merge_layer = concatenate([drop_1, drop_2], axis=1)
lstm_layer = (LSTM(512, 

=2))(merge_layer)
drop_3 =  Dropout(0.5)(lstm_layer)
softmax_1 = Dense(max_span_begin, activation='softmax')(lstm_layer)
softmax_2 = Dense(max_span_end, activation='softmax')(lstm_layer)
model = Model(inputs=[context_input, ques_input], outputs=[softmax_1, softmax_2])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()




Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
c_data (InputLayer)             (None, 700)          0                                            
__________________________________________________________________________________________________
qn_data (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 700, 100)     11961600    c_data[0][0]                     
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 100)     11961600    qn_data[0][0]                    
__________________________________________________________________________________________

In [0]:
model_history = model.fit([c_data[:slce], qn_data[:slce]],
                        [begin_ans[:slce], end_ans[:slce]], verbose=2,
                         batch_size=batch, epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
 - 76s - loss: 14.0440 - dense_1_loss: 6.9920 - dense_2_loss: 7.0520 - dense_1_acc: 0.0278 - dense_2_acc: 0.0040
Epoch 2/10
 - 75s - loss: 13.2684 - dense_1_loss: 6.5954 - dense_2_loss: 6.6730 - dense_1_acc: 0.0279 - dense_2_acc: 0.0066
Epoch 3/10
 - 75s - loss: 13.2309 - dense_1_loss: 6.5761 - dense_2_loss: 6.6548 - dense_1_acc: 0.0279 - dense_2_acc: 0.0071
Epoch 4/10
 - 76s - loss: 13.2146 - dense_1_loss: 6.5669 - dense_2_loss: 6.6477 - dense_1_acc: 0.0279 - dense_2_acc: 0.0065
Epoch 5/10
 - 75s - loss: 13.1966 - dense_1_loss: 6.5573 - dense_2_loss: 6.6393 - dense_1_acc: 0.0279 - dense_2_acc: 0.0063
Epoch 6/10
 - 75s - loss: 13.1846 - dense_1_loss: 6.5524 - dense_2_loss: 6.6322 - dense_1_acc: 0.0279 - dense_2_acc: 0.0069
Epoch 7/10
 - 75s - loss: 13.1806 - dense_1_loss: 6.5503 - dense_2_loss: 6.6303 - dense_1_acc: 0.0279 - dense_2_acc: 0.0071
Epoch 8/10
 - 75s - loss: 13.1781 - 

# PREDICTIONS USING TEST DATA

In [0]:
t_context = h5py.File('/content/drive/My Drive/dl_proj_data/context_test.h5','r')
t_questions = h5py.File('/content/drive/My Drive/dl_proj_data/questions_test.h5','r')
t_answers = h5py.File('/content/drive/My Drive/dl_proj_data/answers_test.h5','r')
t_ans_begin = h5py.File('/content/drive/My Drive/dl_proj_data/begin_test.h5','r')
t_ans_end = h5py.File('/content/drive/My Drive/dl_proj_data/end_test.h5','r')

In [0]:
t_c_data = t_context['context'][:]
t_qn_data = t_questions['questions'][:]
t_ans_data = t_answers['answers'][:]
t_begin_ans = t_ans_begin['begin'][:]
t_end_ans = t_ans_end['end'][:]

In [0]:
index_train = np.load('/content/drive/My Drive/dl_proj_data/indxes.npy', allow_pickle=True)
index_test = np.load('/content/drive/My Drive/dl_proj_data/indxes_test.npy', allow_pickle=True)

In [0]:
predictions = model.predict([t_c_data,t_qn_data], batch_size=128)

In [0]:
print(predictions[0].shape, predictions[1].shape)

(20302, 3126) (20302, 3136)


In [0]:
ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 

In [0]:
for i in range(predictions[0].shape[0]):
	ansBegin[i] = predictions[0][i, :].argmax()
	ansEnd[i] = predictions[1][i, :].argmax()
print(ansBegin.min(), ansBegin.max(), ansEnd.min(), ansEnd.max())

0 0 7 7


In [0]:
import pandas as pd
pd.Series(ansEnd).value_counts()

7    20302
dtype: int64

In [0]:
answers = {}
for i in range(len(vQuestion_id)):
    if ansBegin[i] >= len(vContext[i]):
        answers[vQuestion_id[i]] = ""
    elif ansEnd[i] >= len(vContext[i]):
        answers[vQuestion_id[i]] = vContextOriginal[i][vToken2CharIdx[i][ansBegin[i]]:]
    else:
        answers[vQuestion_id[i]] = vContextOriginal[i][vToken2CharIdx[i][ansBegin[i]]:vToken2CharIdx[i][ansEnd[i]]+len(vContext[i][ansEnd[i]])]